In [2]:
from sentinelhub import SHConfig, BBox, CRS

/home/davide/VirtualEnvs/junction2024.3.9/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
config = SHConfig()
config.sh_client_id = "848a9ae0-b57d-4ad1-98ee-2d72b3882737"
config.sh_client_secret = "TIMjaUmWqncdtvXt1wtBIY4b7HXXn16G"

In [6]:

import datetime
from dateutil.relativedelta import relativedelta
import numpy as np

In [111]:
from sentinelhub import (
    CRS,
    SentinelHubCatalog,
    filter_times,
    BBox,
    DataCollection,
    DownloadRequest,
    MimeType,
    MosaickingOrder,
    SentinelHubDownloadClient,
    bbox_to_dimensions,
    SentinelHubRequest,
    bbox_to_dimensions,
)
import math

resolution = 10

def get_bounding_box_from_center(bounding_boxes_left_corner, bounding_box_side_size_in_metres=100):
      # Earth radius in meters
    R = 6378137.0

    half_side = bounding_box_side_size_in_metres / 2

    lat_offset = half_side / R * (180 / math.pi)

    lon_offset = half_side / (R * math.cos(math.radians(bounding_boxes_left_corner[1]))) * (180 / math.pi)

    top_left = [bounding_boxes_left_corner[1] + lat_offset, bounding_boxes_left_corner[0] - lon_offset]
    bottom_right = [bounding_boxes_left_corner[1] - lat_offset, bounding_boxes_left_corner[0] + lon_offset]


    return BBox(bbox=top_left + bottom_right, crs=CRS.WGS84)

bounding_box = get_bounding_box_from_center([11.414709,46.212418])
betsiboka_size = bbox_to_dimensions(bounding_box, resolution=resolution)


print(betsiboka_size)
# exit(0)

forest_stress = """
//VERSION=3
const moistureRamps = [
    [-0.8, 0x800000],
    [-0.24, 0xff0000],
    [-0.032, 0xffff00],
    [0.032, 0x00ffff],
    [0.24, 0x0000ff],
    [0.8, 0x000080]
  ];

//const viz = new ColorRampVisualizer(moistureRamps);

function setup() {
  return {
    input: ["B8A", "B11", "SCL", "dataMask"],
    output: [
      { id: "default", bands: 1 },
      { id: "index", bands: 1, sampleType: "FLOAT32" },
      { id: "eobrowserStats", bands: 2, sampleType: "FLOAT32" },
      { id: "dataMask", bands: 1 },
    ],
  };
}

function evaluatePixel(samples) {
  let val = index(samples.B8A, samples.B11);
  // The library for tiffs works well only if there is only one channel returned.
  // So we encode the "no data" as NaN here and ignore NaNs on frontend.
  const indexVal = samples.dataMask === 1 ? val : NaN;
  return {
    default: [val],
    index: [indexVal],
    eobrowserStats: [val, isCloud(samples.SCL) ? 1 : 0],
    dataMask: [samples.dataMask],
  };
}

function isCloud(scl) {
  if (scl == 3) {
    // SC_CLOUD_SHADOW
    return false;
  } else if (scl == 9) {
    // SC_CLOUD_HIGH_PROBA
    return true;
  } else if (scl == 8) {
    // SC_CLOUD_MEDIUM_PROBA
    return true;
  } else if (scl == 7) {
    // SC_CLOUD_LOW_PROBA
    return false;
  } else if (scl == 10) {
    // SC_THIN_CIRRUS
    return true;
  } else if (scl == 11) {
    // SC_SNOW_ICE
    return false;
  } else if (scl == 1) {
    // SC_SATURATED_DEFECTIVE
    return false;
  } else if (scl == 2) {
    // SC_DARK_FEATURE_SHADOW
    return false;
  }
  return false;
}

"""

today = datetime.date.today()

one_year_ago = today - relativedelta(years=1)

time_interval = one_year_ago, today

catalog = SentinelHubCatalog(config=config)

search_iterator = catalog.search(
    DataCollection.SENTINEL2_L2A,
    bbox=bounding_box,
    time=time_interval,
    filter="eo:cloud_cover < 30",
    fields={"include": ["id", "properties.datetime"], "exclude": []},
)

results = list(search_iterator)

all_timestamps = search_iterator.get_timestamps()

time_difference = datetime.timedelta(hours=1)

unique_acquisitions = filter_times(all_timestamps, time_difference)

process_requests = []

for timestamp in unique_acquisitions:
    request = SentinelHubRequest(
        evalscript=forest_stress,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L2A,
                time_interval=(timestamp - time_difference, timestamp + time_difference),
            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.PNG)],
        bbox=bounding_box,
        size=betsiboka_size,
        config=config,
    )
    process_requests.append(request)


client = SentinelHubDownloadClient(config=config)

download_requests = [request.download_list[0] for request in process_requests]

data = client.download(download_requests)

images = np.array(data)

print(images.shape)

(10, 14)
(64, 14, 10)


In [112]:
import pandas as pd

In [113]:
dates = pd.to_datetime(unique_acquisitions, format="%Y/%m/%d")
dates = dates.normalize()

In [114]:
dates

DatetimeIndex(['2023-11-13 00:00:00+00:00', '2023-11-23 00:00:00+00:00',
               '2023-11-28 00:00:00+00:00', '2023-12-03 00:00:00+00:00',
               '2023-12-08 00:00:00+00:00', '2023-12-13 00:00:00+00:00',
               '2023-12-18 00:00:00+00:00', '2023-12-23 00:00:00+00:00',
               '2023-12-28 00:00:00+00:00', '2024-01-02 00:00:00+00:00',
               '2024-01-07 00:00:00+00:00', '2024-01-17 00:00:00+00:00',
               '2024-01-22 00:00:00+00:00', '2024-01-27 00:00:00+00:00',
               '2024-02-06 00:00:00+00:00', '2024-02-11 00:00:00+00:00',
               '2024-02-16 00:00:00+00:00', '2024-02-21 00:00:00+00:00',
               '2024-02-26 00:00:00+00:00', '2024-03-02 00:00:00+00:00',
               '2024-03-07 00:00:00+00:00', '2024-03-12 00:00:00+00:00',
               '2024-03-17 00:00:00+00:00', '2024-03-22 00:00:00+00:00',
               '2024-03-27 00:00:00+00:00', '2024-04-01 00:00:00+00:00',
               '2024-04-06 00:00:00+00:00', '2024-0

In [126]:
collected_data = pd.DataFrame(data=images.reshape(images.shape[0], -1), index=dates)

In [ ]:
min_date = collected_data.index.min()
print(min_date)
dt = datetime.datetime.now()
dt = dt.replace(tzinfo=datetime.timezone.utc)
max_date = dt
daterange = pd.date_range(min_date, max_date, normalize=True)
collected_data = collected_data.resample('1W').mean().ffill()

2023-11-19 00:00:00+00:00


In [135]:
collected_data.tail(n=15)

,0,1,2,3,4,5,6,7,8,9,...,130,131,132,133,134,135,136,137,138,139
2024-08-04 00:00:00+00:00,0.5,0.5,0.0,0.0,0.0,0.0,2.0,2.0,3.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.5,3.5,0.0,0.0
2024-08-11 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024-08-18 00:00:00+00:00,6.0,6.0,10.0,10.0,10.0,10.0,12.0,12.0,11.0,11.0,...,1.0,1.0,5.0,5.0,8.0,8.0,10.0,10.0,4.0,4.0
2024-08-25 00:00:00+00:00,10.0,10.0,0.0,0.0,0.0,0.0,7.0,7.0,0.0,0.0,...,2.0,2.0,5.0,5.0,4.0,4.0,0.0,0.0,3.0,3.0
2024-09-01 00:00:00+00:00,0.0,0.0,3.0,3.0,0.0,0.0,1.0,1.0,2.0,2.0,...,2.0,2.0,0.0,0.0,1.0,1.0,2.0,2.0,1.0,1.0
2024-09-08 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.5,0.5,...,0.0,0.0,0.0,0.0,4.5,4.5,0.0,0.0,0.0,0.0
2024-09-15 00:00:00+00:00,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0
2024-09-22 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024-09-29 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024-10-06 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [134]:
min_date

Timestamp('2023-11-19 00:00:00+0000', tz='tzutc()')

In [67]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [136]:
three_weeks = datetime.timedelta(days=21)
three_weeks_from_now = dt + three_weeks
max_date = collected_data.index.max()

In [161]:
predictions = []
n_weeks = 0

for column in collected_data:
    area_measures = ExponentialSmoothing(collected_data[column]).fit()
    area_prediction = area_measures.predict(start=max_date, end=three_weeks_from_now)
    n_weeks = len(area_prediction)
    predictions.append(area_prediction)

predictions = np.array(predictions).reshape((n_weeks, 14, 10))

In [164]:
print(predictions[0])

[[2.65384623 2.65384619 2.65384619 2.65384619 2.65384623 2.65384619
  2.65384619 2.65384619 3.53846156 3.5384617 ]
 [3.5384617  3.5384617  3.53846156 3.5384617  3.5384617  3.5384617
  2.94230787 2.94230782 2.94230782 2.94230782]
 [2.94230787 2.94230782 2.94230782 2.94230782 2.67307677 2.67307683
  2.67307683 2.67307683 2.67307677 2.67307683]
 [2.67307683 2.67307683 2.73076927 2.73076923 2.73076923 2.73076923
  2.73076927 2.73076923 2.73076923 2.73076923]
 [2.65384623 2.65384619 2.65384619 2.65384619 2.65384623 2.65384619
  2.65384619 2.65384619 3.53846156 3.5384617 ]
 [3.5384617  3.5384617  3.53846156 3.5384617  3.5384617  3.5384617
  2.94230787 2.94230782 2.94230782 2.94230782]
 [2.94230787 2.94230782 2.94230782 2.94230782 2.67307677 2.67307683
  2.67307683 2.67307683 2.67307677 2.67307683]
 [2.67307683 2.67307683 2.73076927 2.73076923 2.73076923 2.73076923
  2.73076927 2.73076923 2.73076923 2.73076923]
 [2.40384625 2.40384626 2.40384626 2.40384626 2.40384625 2.40384626
  2.40384626 2